Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from itertools import product
import gc

from google.colab import drive
drive.mount('/content/drive')

#import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import TimeSeriesSplit
#from sklearn.metrics import mean_squared_error

!pip install tslearn
from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 13.2 MB/s eta 0:00:00


Carga de las bases

In [ ]:
archivo_sellin='/content/drive/MyDrive/sell-in.txt.gz'
archivo_productos='https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt'
archivo_stock='https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt'
archivo_lista_a_predecir='https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt'

df_sellin=pd.read_csv(archivo_sellin,compression='gzip',sep='\t')
df_productos=pd.read_csv(archivo_productos,sep='\t')
df_stock=pd.read_csv(archivo_stock,sep='\t')
df_lista=pd.read_csv(archivo_lista_a_predecir,sep='\t')

def ver_df(df):
  separador='-'*80
  print(separador)
  print(df.head(5))
  print(separador)

dfs=[df_sellin,df_productos,df_stock]
dfs_name=['df_sellin','df_productos','df_stock']

for dataframe in dfs:
  ver_df(dataframe)

--------------------------------------------------------------------------------
   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
    cat1      cat2         cat3   brand  sku_size  product_id  \
0  FOODS  ADEREZOS  Aji Picante

In [ ]:
# parámetros de entrenamiento
ultimo_mes=pd.to_datetime('2019-12-01') # para elegir contra kaggle o diciembre
fecha_test=ultimo_mes
cantidad_meses_validation=2
inicio_fecha_validation=fecha_test-pd.DateOffset(months=cantidad_meses_validation+1)
fin_fecha_validation=inicio_fecha_validation+pd.DateOffset(months=cantidad_meses_validation)-pd.DateOffset(days=1)
hasta_fecha_train=inicio_fecha_validation-pd.DateOffset(days=1)
fecha_filtro_str=ultimo_mes.strftime('%Y%m%d')

n_clusters=5

#**DF**

In [ ]:
archivo_con_combinaciones='df_sellin_combinaciones_agregadas.parquet'

if os.path.exists(archivo_con_combinaciones):
  print('Cargando desde disco...')
  df_sellin=pd.read_parquet(archivo_con_combinaciones)
else:
  print('Procesando desde cero')

  df_sellin['periodo']=pd.to_datetime(df_sellin['periodo'],format='%Y%m')

  # agrega 0 en combinaciones faltantes

  # rango de meses
  meses=pd.date_range(start=df_sellin['periodo'].min(),end=df_sellin['periodo'].max(),freq='MS')
  # clientes unicos
  clientes=df_sellin['customer_id'].unique()
  # productos unicos
  productos=df_sellin['product_id'].unique()
  # combinaciones reales
  combinaciones_reales=df_sellin[['customer_id','product_id']].drop_duplicates()
  combinaciones_reales['control']=combinaciones_reales['customer_id'].astype(str)+'_'+combinaciones_reales['product_id'].astype(str)
  # combinaciones
  combinaciones=pd.DataFrame(list(product(clientes,productos,meses)),columns=['customer_id','product_id','periodo'])
  # merge
  df_sellin=pd.merge(combinaciones,df_sellin,on=['customer_id','product_id','periodo'],how='left')
  # rellena Nan con 0
  df_sellin['cust_request_qty']=df_sellin['cust_request_qty'].fillna(0)
  df_sellin['cust_request_tn']=df_sellin['cust_request_tn'].fillna(0)
  df_sellin['tn']=df_sellin['tn'].fillna(0)
  df_sellin['plan_precios_cuidados']=df_sellin['plan_precios_cuidados'].fillna(0)
  # borra filas vacías
  df_sellin['control_2']=df_sellin['customer_id'].astype(str)+'_'+df_sellin['product_id'].astype(str)
  df_sellin['flag']=df_sellin['control_2'].isin(combinaciones_reales['control'])
  df_sellin=df_sellin[df_sellin['flag']]
  df_sellin=df_sellin.drop(columns=['control_2','flag'])
  # ordena
  df_sellin=df_sellin.sort_values(by=['customer_id','product_id','periodo']).reset_index(drop=True)

  # merge con df_productos
  df_sellin=df_sellin.merge(df_productos,on='product_id',how='left')
  df_sellin.to_parquet(archivo_con_combinaciones)

Procesando desde cero


In [ ]:
# media movil

archivo_medias='df_medias.parquet'
archivo_medias_kaggle='df_medias_kaggle.parquet'

if os.path.exists(archivo_medias):
  print('Cargando desde disco...')
  df_medias=pd.read_parquet(archivo_medias)
  df_medias_kaggle=pd.read_parquet(archivo_medias_kaggle)
else:
  df_medias=df_sellin[df_sellin['product_id'].isin(df_lista['product_id'])]
  df_medias=df_medias[['product_id','periodo','tn']]
  df_medias=df_medias.sort_values(by=['product_id','periodo'])
  df_medias=df_medias.groupby(['product_id','periodo'],as_index=False)['tn'].sum()
  df_medias['y']=(df_medias.groupby('product_id')['tn'].transform(lambda x: x.rolling(window=12).mean()))
  #df_medias['y']=df_medias.groupby('product_id')['tn'].transform(lambda x: x.ewm(span=18,adjust=True).mean())
  df_medias_kaggle=df_medias[df_medias['periodo']==fecha_test][['product_id','y']]

  df_medias.to_parquet(archivo_medias)
  df_medias_kaggle.to_parquet(archivo_medias_kaggle)

df_medias_kaggle.to_csv('kaggle_media12.csv',index=False)

In [ ]:
# agrupa, pivotea y filtra, quedando con una fila por mes y una columna por cada producto a predecir
df_sellin=df_sellin[['product_id','periodo','tn']]
df_sellin=df_sellin.groupby(['periodo','product_id'],as_index=False)['tn'].sum()
df_sellin_pivot=df_sellin.pivot(index='periodo',columns='product_id',values='tn')
columnas_conservar=df_lista['product_id'].tolist()
df_sellin_pivot=df_sellin_pivot.loc[:,df_sellin_pivot.columns.isin(columnas_conservar)]
#validos=df_sellin_pivot.apply(lambda col: col.ne(0).idxmax()==df_sellin_pivot.index[0])
#productos_red=validos[validos].index.tolist()
#productos_media=validos[~validos].index.tolist()
#df_sellin_pivot=df_sellin_pivot[productos_red]
df_sellin_pivot.tail(5)

product_id,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-08-01,1261.34529,813.78215,635.59563,482.13372,536.66800,262.73593,307.82899,233.00983,520.41758,199.86233,...,0.02117,0.13446,0.02965,0.03106,0.02258,0.03388,0.01593,0.01480,0.04054,0.01265
2019-09-01,1660.00561,1090.18771,967.77116,786.17140,879.52808,409.95501,369.74894,330.56343,558.45719,524.94628,...,0.01411,0.14427,0.00423,0.00564,0.01552,0.01128,0.01707,0.02844,0.01830,0.01856
2019-10-01,1561.50552,1979.53635,1081.36645,1064.69633,996.78275,528.32630,445.34884,452.77197,556.15182,448.82078,...,0.01270,0.10025,0.00847,0.01975,0.01693,0.01552,0.10921,0.11831,0.09676,0.02079
2019-11-01,1397.37231,1423.57739,948.29393,723.94206,606.91173,399.61420,357.85913,396.49833,711.89025,470.96658,...,0.02964,0.11982,0.02682,0.02965,0.02682,0.03247,0.06600,0.06713,0.04052,0.03341
2019-12-01,1504.68856,1087.30855,892.50129,637.90002,593.24443,417.23228,390.43432,195.36854,495.03574,359.59998,...,0.01129,0.08560,0.01271,0.01412,0.01834,0.01270,0.05007,0.05121,0.01569,0.00892


In [ ]:
# features temporales
df_temporales=pd.DataFrame(index=df_sellin_pivot.index)
mes=df_temporales.index.month
df_temporales['mes_sin']=np.sin(2*np.pi*mes/12)
df_temporales['mes_cos']=np.cos(2*np.pi*mes/12)
alfa=2.5
meses_rbf=[1,2,3,4,5,6,7,8,9,10,11,12]
for mu in meses_rbf:
  delta=np.minimum(np.abs(mes-mu),12-np.abs(mes-mu))
  df_temporales[f'mes_rbf_{mu:02d}']=np.exp(-0.5*(delta**2)/alfa)
numero_filas_hasta_train=df_temporales.index.to_series().le(hasta_fecha_train).sum()-1
df_temporales['tendencia']=(np.arange(len(df_sellin_pivot)))/numero_filas_hasta_train
df_temporales

def asignar_features_temporales(df_1,df_temporales):
  df_1=df_1.merge(df_temporales,left_index=True,right_index=True,how='left')
  return df_1

In [ ]:
# clase t2
df_target=df_sellin_pivot.shift(-2)
df_target.columns=[f'{col}_t2' for col in df_target.columns]
df_target.tail(10)

,20001_t2,20002_t2,20003_t2,20004_t2,20005_t2,20006_t2,20007_t2,20008_t2,20009_t2,20010_t2,...,21248_t2,21252_t2,21256_t2,21259_t2,21262_t2,21263_t2,21265_t2,21266_t2,21267_t2,21276_t2
periodo,,,,,,,,,,,,,,,,,,,,,
2019-03-01,1629.78233,1034.98927,590.12515,603.31081,897.26297,527.68846,394.67651,486.36682,578.48564,408.41680,...,0.01553,0.47194,0.01553,0.01976,0.01130,0.02258,0.17635,0.17634,0.24451,0.09283
2019-04-01,1109.93769,928.36431,662.38654,667.19411,876.39696,458.04180,536.13689,567.42091,610.39590,600.25060,...,0.00988,0.07824,0.00988,0.00988,0.01270,0.00988,0.05121,0.05235,0.05882,0.04086
2019-05-01,1678.99318,1066.44999,715.20314,521.71519,745.74978,343.11053,573.37257,524.04994,716.07987,463.91662,...,0.02116,0.02691,0.02822,0.04657,0.03953,0.03387,0.02959,0.05916,0.07452,0.00223
2019-06-01,1261.34529,813.78215,635.59563,482.13372,536.66800,262.73593,307.82899,233.00983,520.41758,199.86233,...,0.02117,0.13446,0.02965,0.03106,0.02258,0.03388,0.01593,0.01480,0.04054,0.01265
2019-07-01,1660.00561,1090.18771,967.77116,786.17140,879.52808,409.95501,369.74894,330.56343,558.45719,524.94628,...,0.01411,0.14427,0.00423,0.00564,0.01552,0.01128,0.01707,0.02844,0.01830,0.01856
2019-08-01,1561.50552,1979.53635,1081.36645,1064.69633,996.78275,528.32630,445.34884,452.77197,556.15182,448.82078,...,0.01270,0.10025,0.00847,0.01975,0.01693,0.01552,0.10921,0.11831,0.09676,0.02079
2019-09-01,1397.37231,1423.57739,948.29393,723.94206,606.91173,399.61420,357.85913,396.49833,711.89025,470.96658,...,0.02964,0.11982,0.02682,0.02965,0.02682,0.03247,0.06600,0.06713,0.04052,0.03341
2019-10-01,1504.68856,1087.30855,892.50129,637.90002,593.24443,417.23228,390.43432,195.36854,495.03574,359.59998,...,0.01129,0.08560,0.01271,0.01412,0.01834,0.01270,0.05007,0.05121,0.01569,0.00892
2019-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# division en training, validation y test para escalado

features_train=df_sellin_pivot[df_sellin_pivot.index<=hasta_fecha_train]
features_validation=df_sellin_pivot[(df_sellin_pivot.index>=inicio_fecha_validation)&(df_sellin_pivot.index<=fin_fecha_validation)]
features_test=df_sellin_pivot[df_sellin_pivot.index==fecha_test]

target_train=df_target[df_target.index<=hasta_fecha_train]
target_validation=df_target[(df_target.index>=inicio_fecha_validation)&(df_target.index<=fin_fecha_validation)]
target_test=df_target[df_target.index==fecha_test]

In [ ]:
# escalado en features (no target)
scaler=StandardScaler()
scaler.fit(features_train)
features_train_scaled=pd.DataFrame(scaler.transform(features_train),index=features_train.index,columns=features_train.columns)
features_validation_scaled=pd.DataFrame(scaler.transform(features_validation),index=features_validation.index,columns=features_validation.columns)
features_test_scaled=pd.DataFrame(scaler.transform(features_test),index=features_test.index,columns=features_test.columns)
features_escaladas=pd.concat([features_train_scaled,features_validation_scaled,features_test_scaled])

In [ ]:
# features: medias móviles

windows=[12]

features_fe=features_escaladas.copy()
columnas_originales=features_escaladas.columns

mm_dfs=[]
for window in windows:
  mm=features_escaladas.rolling(window=window).mean()
  mm.columns=[f'mm{window}_{col}' for col in columnas_originales]
  mm_dfs.append(mm)

features_fe=pd.concat([features_escaladas]+mm_dfs,axis=1)

features_fe=features_fe.filter(like='mm',axis=1)
features_fe.tail(10)

,mm12_20001,mm12_20002,mm12_20003,mm12_20004,mm12_20005,mm12_20006,mm12_20007,mm12_20008,mm12_20009,mm12_20010,...,mm12_21248,mm12_21252,mm12_21256,mm12_21259,mm12_21262,mm12_21263,mm12_21265,mm12_21266,mm12_21267,mm12_21276
periodo,,,,,,,,,,,,,,,,,,,,,
2019-02-01,0.493307,0.703736,-0.004398,0.006347,-0.253669,-0.078926,-0.151005,-0.180141,0.085152,-0.578827,...,0.478596,-0.374217,0.449046,0.445292,0.466662,0.453833,-0.293888,-0.306778,-0.396039,-0.371533
2019-03-01,0.388132,0.742357,-0.043064,0.055371,-0.281517,-0.267274,-0.407038,-0.358514,0.142760,-0.625255,...,0.523342,-0.086202,0.494172,0.496129,0.518471,0.503476,-0.274771,-0.286287,-0.095032,-0.045994
2019-04-01,0.496078,0.837754,-0.098138,0.001118,-0.231626,-0.221707,-0.471583,-0.523382,0.214087,-0.527723,...,0.549247,0.089880,0.508136,0.512010,0.529986,0.517203,0.162105,0.147995,0.076423,0.224371
2019-05-01,0.587555,0.815129,-0.151586,-0.013142,-0.130684,-0.398455,-0.603183,-0.650095,0.355595,-0.463464,...,0.562199,0.443937,0.519954,0.526832,0.539201,0.534095,0.373733,0.359674,0.417507,0.471083
2019-06-01,0.576429,0.780245,-0.151131,0.069041,-0.003116,-0.403749,-0.701939,-0.642735,0.516278,-0.329971,...,0.570440,0.502633,0.527473,0.534244,0.549558,0.541486,0.435187,0.422515,0.499559,0.579675
2019-07-01,0.633236,0.809699,-0.134902,0.019694,0.091329,-0.493847,-0.751375,-0.644070,0.768999,-0.283171,...,0.588088,0.522822,0.548948,0.569178,0.581795,0.566824,0.470696,0.493530,0.603512,0.585602
2019-08-01,0.486273,0.694560,-0.223061,-0.150512,-0.039905,-0.678642,-0.908704,-0.828396,0.762049,-0.419003,...,0.605744,0.623695,0.571511,0.592477,0.600209,0.592169,0.489813,0.511296,0.660064,0.619221
2019-09-01,0.546552,0.739528,-0.207808,-0.211467,0.005785,-0.714289,-0.934971,-0.889339,0.739867,-0.403275,...,0.617513,0.731929,0.574730,0.596708,0.612866,0.600607,0.510298,0.545436,0.685592,0.668548
2019-10-01,0.346732,0.938330,-0.271642,-0.115917,0.045767,-0.706346,-0.976598,-0.934981,0.708873,-0.461332,...,0.244132,0.807138,0.172810,0.216385,0.239714,0.218164,0.641355,0.687455,0.820570,0.723801


In [ ]:
# division training, validation y test para modelado
X_train=features_fe[features_fe.index<=hasta_fecha_train]
X_train=X_train.dropna()
X_validation=features_fe[(features_fe.index>=inicio_fecha_validation)&(features_fe.index<=fin_fecha_validation)]
X_test=features_fe[features_fe.index==fecha_test]
target_train=target_train.loc[X_train.index]

In [ ]:
# dtw
df_clustering=X_train.filter(regex=r'^mm\d+_',axis=1).T
data=df_clustering.values[:,:,None]
scaler=TimeSeriesScalerMeanVariance()
data_scaled=scaler.fit_transform(data)
model=TimeSeriesKMeans(n_clusters=n_clusters,metric='dtw',random_state=13)
labels=model.fit_predict(data_scaled)
clusters=pd.DataFrame({'product_id':df_clustering.index,'cluster':labels})

In [ ]:
# modelo red densa

semillas=[7,13,1007,164,87]
clusters_encontrados=clusters['cluster'].unique()
final=pd.DataFrame()

for c in clusters_encontrados:
  y_pred_lista=[]
  n=0
  cluster_analizado_X=clusters[clusters['cluster']==c]['product_id'].tolist()
  cluster_analizado_target=[col.replace('mm12_','')+'_t2' for col in cluster_analizado_X]
  X_train_cluster=X_train[cluster_analizado_X]
  target_train_cluster=target_train[cluster_analizado_target]
  X_validation_cluster=X_validation[cluster_analizado_X]
  target_validation_cluster=target_validation[cluster_analizado_target]
  X_test_cluster=X_test[cluster_analizado_X]
  X_train_cluster=asignar_features_temporales(X_train_cluster,df_temporales)
  X_validation_cluster=asignar_features_temporales(X_validation_cluster,df_temporales)
  X_test_cluster=asignar_features_temporales(X_test_cluster,df_temporales)

  for seed in semillas:
    n=n+1
    tf.random.set_seed(seed)
    np.random.seed(seed)

    input_dim=X_train_cluster.shape[1]
    output_dim=target_train_cluster.shape[1]

    model=Sequential([
      Input(shape=(input_dim,)),
       Dense(64,activation='relu'),
       Dense(32,activation='relu'),
        Dense(output_dim)
   ])

    model.compile(optimizer='adam',loss='mae',metrics=['mape'])

    early_stop=EarlyStopping(patience=20,restore_best_weights=True)

    history=model.fit(
        X_train_cluster,target_train_cluster,
       validation_data=(X_validation_cluster,target_validation_cluster),
        epochs=200,
        batch_size=16,
        callbacks=[early_stop],
       verbose=0
    )
    print(f'Entrenamiento {n} cluster {c} terminado')

    y_pred_for=model.predict(X_test_cluster)[0]
    y_pred_lista.append(y_pred_for)
    print(f'Predicción {n} cluster {c} terminada')

  y_pred_cluster=np.array(y_pred_lista)
  y_pred_cluster=y_pred_cluster.mean(axis=0)
  y_pred_cluster=np.maximum(y_pred_cluster,0)
  final_cluster=pd.DataFrame({'product_id':cluster_analizado_X,'y':y_pred_cluster})
  final=pd.concat([final,final_cluster])

final.sort_values('product_id',inplace=True)

print('Listo')

Entrenamiento 1 cluster 3 terminado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicción 1 cluster 3 terminada
Entrenamiento 1 cluster 0 terminado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicción 1 cluster 0 terminada
Entrenamiento 1 cluster 1 terminado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicción 1 cluster 1 terminada
Entrenamiento 1 cluster 2 terminado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicción 1 cluster 2 terminada


Entrenamiento 1 cluster 4 terminado
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicción 1 cluster 4 terminada
Listo


In [ ]:
# prediccion
product_ids=df_sellin_pivot.columns.tolist()
final_red=pd.DataFrame({'product_id':product_ids})

final['product_id']=final['product_id'].str.replace('mm12_','',regex=False).astype(int)
final_red=final_red.merge(final,on='product_id',how='left')
final_red.to_csv('kaggle_red.csv',index=False)

In [ ]:
# regresión
lista=df_lista['product_id'].unique()
df_sellin_regre=df_sellin[df_sellin['product_id'].isin(lista)].copy()
df_sellin_regre['y']=df_sellin_regre.groupby('product_id')['tn'].shift(-2)
#df_sellin_regre['tendencia']=df_sellin_regre.groupby('product_id').cumcount()+1
#df_sellin_regre['mes']=df_sellin_regre['periodo'].dt.month
#df_sellin_regre['mes_sin']=np.sin(2*np.pi*df_sellin_regre['mes']/12)
#df_sellin_regre['mes_cos']=np.cos(2*np.pi*df_sellin_regre['mes']/12)
#df_sellin_regre['media_movil_12']=df_sellin_regre.groupby('product_id')['tn'].transform(lambda x: x.rolling(window=12).mean())
lags_regre=range(1,19)
for lag in lags_regre:
  df_sellin_regre[f'lag_{lag}']=df_sellin_regre.groupby('product_id')['tn'].transform(lambda x: x.shift(lag))

X=df_sellin_regre.drop(columns=['product_id','y'])
y=df_sellin_regre['y']
mask_train=df_sellin_regre['periodo']<fin_fecha_validation
mask_pred=df_sellin_regre['periodo']==fecha_test
X_train=X[mask_train]
y_train=y[mask_train]
X_pred=X[mask_pred]
X_train=X_train.dropna()
X_train=X_train.drop(columns='periodo')
X_pred=X_pred.drop(columns='periodo')
y_train=y_train.loc[X_train.index]

model_regre=LinearRegression()
model_regre.fit(X_train,y_train)
y_pred=model_regre.predict(X_pred)
final_regre=pd.DataFrame({'product_id':lista,'y':y_pred})
final_regre.to_csv('kaggle_regre.csv',index=False)

In [ ]:
# ensemble
ensemble=final_red.merge(df_medias_kaggle,on='product_id',how='left')
ensemble=ensemble.rename(columns={'y_x':'y_red','y_y':'y_media12'})
ensemble=ensemble.merge(final_regre,on='product_id',how='left')
ensemble=ensemble.rename(columns={'y':'y_regre'})
diciembre=df_sellin_pivot.iloc[-1].reset_index()
diciembre.columns=['product_id','dic']
ensemble=ensemble.merge(diciembre,on='product_id',how='left')
#ensemble['y']=np.where(ensemble['dic']>25,ensemble['y_red'],ensemble['y_media12'])
ensemble['y']=ensemble['y_red']*0.8+ensemble['y_regre']*0.2+ensemble['y_media12']*0
ensemble=ensemble[['product_id','y']]
ensemble.to_csv('kaggle_e.csv',index=False)